### Headline network for left and right news media

In [1]:
# Importing modules
import re, nltk, numpy, matplotlib
import pandas as pd
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer, PorterStemmer, SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer 
from sklearn.decomposition import LatentDirichletAllocation
from gensim import corpora
from collections import Counter
import itertools
from itertools import combinations
import matplotlib.pyplot as plt
from collections import defaultdict
from nltk.tokenize import word_tokenize
import operator
import itertools
from itertools import combinations
# Load matplotlib
import matplotlib.pyplot as plt
from collections import defaultdict


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\2146806A\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\2146806A\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\2146806A\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load categorised data by media ideological affiliation
df = pd.read_csv('C:/Users/2146806A/OneDrive - University of Glasgow/University of Glasgow/Amsterdam Visit/GITHUB FILES/Data/preprocessed.csv', encoding='latin-1')
df.head()  


,date,maintext,title,source,media_name,ideology,Congress,maintext_no_fullstop,text_lemma,title_lemma
0,03/01/2013,greg gutfeld cohost america. greg gutfeld andr...,reagan era al jazeera buys current tv,C:/Users/2146806A/OneDrive - University of Gla...,Fox News,right,113th,greg gutfeld cohost america greg gutfeld andre...,greg gutfeld cohost america greg gutfeld andre...,reagan era al jazeera buy current tv
1,11/01/2013,latest threat leadership easily speaker house ...,job safe straus challenge gop,C:/Users/2146806A/OneDrive - University of Gla...,The New York Times,left,113th,latest threat leadership easily speaker house ...,late threat leadership easily speaker house jo...,job safe straus challenge gop
2,18/01/2013,story highlights mills remembers march selma m...,years mlk march voting rights vulnerable,https://edition.cnn.com/2013/01/18/opinion/mil...,CNN,left,113th,story highlights mills remembers march selma m...,story highlight mill remember march selma mont...,year mlk march vote right vulnerable
3,20/01/2013,barack michelle obama spent thousand days disp...,change years friends shifts obamas,C:/Users/2146806A/OneDrive - University of Gla...,The New York Times,left,113th,barack michelle obama spent thousand days disp...,barack michelle obama spend thousand day displ...,change year friend shift obama
4,21/01/2013,sean hannity host hannity moments ago presiden...,interview karl rove,C:/Users/2146806A/OneDrive - University of Gla...,Fox News,right,113th,sean hannity host hannity moments ago presiden...,sean hannity host hannity moment ago president...,interview karl rove


In [49]:
nodes_df = pd.read_csv('C:/Users/2146806A/OneDrive - University of Glasgow/University of Glasgow/Amsterdam Visit/GITHUB FILES/Data/shifterator_scores.csv', encoding='latin-1')

In [4]:
df['title'] = df['title'].str.replace("'", "")

#### Split corpus into right and left news media

In [5]:
import re
import string
from tqdm import tqdm
from nltk.tokenize import sent_tokenize

In [6]:
# left news media
left = df[df['ideology'].isin(['left'])]

# right news media
right = df[df['ideology'].isin(['right'])]

#### Split headlines into a list

In [11]:
# we want to find co-occurrences of words in sentences so we convert the text_sentence column into a list
right_list = right['title'].tolist()
left_list = left['title'].tolist()

In [135]:
sentences_left = []
for article in left_list:
    if isinstance(article, str) and article.strip():
        article_sentences = sent_tokenize(article)
        sentences_left.extend(article_sentences)

df_sentences_left= pd.DataFrame({'sentences': sentences_left})

sentences_right = []
for article in right_list:
    if isinstance(article, str) and article.strip():
        article_sentences = sent_tokenize(article)
        sentences_right.extend(article_sentences)

df_sentences_right= pd.DataFrame({'sentences': sentences_right})

## Generating edge file

In [10]:
# we create a list of individual sentences
sentences_left = df_sentences_left['sentences'].tolist()
sentences_right = df_sentences_right['sentences'].tolist()

NameError: name 'df_sentences_left' is not defined

In [ ]:
tokenized_left = [word_tokenize(e) for e in left_list]
tokenized_right = [word_tokenize(e) for e in right_list]

In [19]:
# tokens in every list item (sentence)
# Preprocess the lists
tokenized_left = []
for item in left_list:
    if isinstance(item, str) and item.strip():  # Check if it's a non-empty string
        tokenized_left.append(word_tokenize(item))
    else:
        # Handle non-string or empty string elements (you can skip or handle them differently)
        print(f"Ignoring item: {item}")
tokenized_right = []
for item in right_list:
    if isinstance(item, str) and item.strip():  # Check if it's a non-empty string
        tokenized_right.append(word_tokenize(item))
    else:
        # Handle non-string or empty string elements (you can skip or handle them differently)
        print(f"Ignoring item: {item}")


Ignoring item: nan
Ignoring item: nan
Ignoring item: nan
Ignoring item: nan
Ignoring item: nan
Ignoring item: nan
Ignoring item: nan
Ignoring item: nan
Ignoring item: nan
Ignoring item: nan


We then create an edge list dictionary. For every word that occurs with another word we give it a value of 1. <br> 
So it looks like this: <br> 
 {('ways', 'league'): 2, <br> 
   ('ways', 'women'): 2, <br> 
  ('ways', 'voters'): 3, ... <br> 

In [12]:
#tokenized_left = [word_tokenize(e) if isinstance(e, str) else '' for e in left_list]
#tokenized_right = [word_tokenize(e) if isinstance(e, str) else '' for e in right_list]

In [20]:
# edgelist dictionary for right
edgelist_right = defaultdict(int)
for text in tokenized_right:
    for node1, node2 in combinations(text, 2):
        if node1 != node2:  # Skip pairs with the same node
            edgelist_right[(node1, node2)] += 1

# left
edgelist_left = defaultdict(int)
for text in tokenized_left:
    for node1, node2 in combinations(text, 2):
        if node1 != node2:  # Skip pairs with the same node
            edgelist_left[(node1, node2)] += 1

In [21]:
# we sort the edglist by value so that we can see what is the maximum which can inform us about where to make the cut
sorted_edges_r = sorted(edgelist_right.items(), key=operator.itemgetter(1), reverse = True)
# print the list from top down to get an idea about how frequent are the occurrences to be able to filter later on
sorted_edges_r

[(('voter', 'identification'), 181),
 (('president', 'trump'), 109),
 (('president', 'biden'), 107),
 (('biden', 'president'), 105),
 (('interview', 'senator'), 91),
 (('biden', 'covid'), 85),
 (('biden', 'border'), 77),
 (('interview', 'rep'), 74),
 (('covid', 'biden'), 68),
 (('biden', 'interview'), 67),
 (('biden', 'administration'), 65),
 (('trump', 'president'), 61),
 (('identification', 'law'), 60),
 (('biden', 'us'), 55),
 (('trump', 'biden'), 53),
 (('election', 'integrity'), 51),
 (('biden', 'crisis'), 51),
 (('voter', 'law'), 50),
 (('democrats', 'voting'), 50),
 (('biden', 'people'), 50),
 (('democrats', 'biden'), 48),
 (('left', 'biden'), 47),
 (('left', 'covid'), 47),
 (('voter', 'laws'), 46),
 (('biden', 'media'), 46),
 (('biden', 'voting'), 45),
 (('gop', 'biden'), 45),
 (('voting', 'bill'), 44),
 (('identification', 'laws'), 43),
 (('covid', 'president'), 42),
 (('biden', 'democrats'), 42),
 (('gop', 'president'), 42),
 (('voting', 'law'), 40),
 (('biden', 'bill'), 40),

In [22]:
sorted_edges_l = sorted(edgelist_left.items(), key=operator.itemgetter(1), reverse = True)
# print the list from top down to get an idea about how frequent are the occurrences to be able to filter later on
sorted_edges_l

[(('voting', 'rights'), 209),
 (('voter', 'identification'), 208),
 (('identification', 'law'), 126),
 (('voter', 'law'), 117),
 (('voter', 'fraud'), 78),
 (('voter', 'laws'), 51),
 (('texas', 'voter'), 50),
 (('texas', 'law'), 49),
 (('identification', 'laws'), 47),
 (('voting', 'law'), 46),
 (('texas', 'identification'), 46),
 (('voter', 'suppression'), 46),
 (('georgia', 'voting'), 45),
 (('gop', 'voting'), 42),
 (('democrats', 'voting'), 40),
 (('voting', 'restrictions'), 33),
 (('voting', 'laws'), 32),
 (('voting', 'bill'), 31),
 (('civil', 'rights'), 30),
 (('north_carolina', 'voter'), 30),
 (('georgia', 'law'), 28),
 (('texas', 'voting'), 27),
 (('opinion', 'voting'), 27),
 (('trump', 'voter'), 27),
 (('election', 'fraud'), 23),
 (('north_carolina', 'identification'), 23),
 (('court', 'law'), 23),
 (('trump', 'fraud'), 22),
 (('republicans', 'voting'), 22),
 (('north_carolina', 'law'), 21),
 (('justice', 'dept'), 20),
 (('black', 'voters'), 20),
 (('democrats', 'rights'), 20),
 

In [24]:
# we filter the number of edges based on weight (aka the frequency of co-occurrence) 
filtered_r = {k: v for k, v in edgelist_right.items() if v > 5}
len(filtered_r)

2439

In [25]:
# we filter the number of edges based on weight (aka the frequency of co-occurrence) 
filtered_l = {k: v for k, v in edgelist_left.items() if v > 5}
len(filtered_l)

363

In [26]:
# we create a dataframe from the edgelist dictionary wXXith a weight column denoting the number of co=occurrences of a speciic pairing
edges_right = pd.DataFrame.from_dict({'Weight': list(filtered_r.values()), 'Edge': list(filtered_r.keys())})
edges_right.head(20)

,Weight,Edge
0,15,"(interview, jordan)"
1,8,"(interview, house)"
2,91,"(interview, senator)"
3,7,"(house, interview)"
4,8,"(senator, scott)"
5,30,"(texas, voter)"
6,25,"(texas, identification)"
7,22,"(texas, law)"
8,181,"(voter, identification)"
9,50,"(voter, law)"


In [27]:
# we create a dataframe from the edgelist dictionary wXXith a weight column denoting the number of co=occurrences of a speciic pairing
edges_left = pd.DataFrame.from_dict({'Weight': list(filtered_l.values()), 'Edge': list(filtered_l.keys())})
edges_left.head(20)

,Weight,Edge
0,6,"(years, voting)"
1,7,"(years, rights)"
2,8,"(march, voting)"
3,7,"(march, rights)"
4,209,"(voting, rights)"
5,12,"(north, dakota)"
6,9,"(early, voting)"
7,7,"(supreme_court, vra)"
8,7,"(supreme_court, case)"
9,46,"(voting, law)"


In [28]:
# we extract the words from the column edge so that we have a source and target word
weight_r = edges_right['Weight']
source_r = [x[0] for x in edges_right['Edge']]
target_r = [x[1] for x in edges_right['Edge']]

weight_r = weight_r.to_list()

# we create a dataframe of edges
df_edges_r = {
    'col1': source_r,
    'col2': target_r,
    'col3': weight_r,
}

df_edges_r = pd.DataFrame(df_edges_r)
df_edges_r.rename(columns={'col1': 'Source', 'col2': 'Target', 'col3':'Weight'}, inplace=True)

df_edges_r = df_edges_r.assign(Type='Undirected')

In [29]:
# we extract the words from the column edge so that we have a source and target word
weight_l = edges_left['Weight']
source_l = [x[0] for x in edges_left['Edge']]
target_l = [x[1] for x in edges_left['Edge']]

weight_l = weight_l.to_list()

# we create a dataframe of edges
df_edges_l = {
    'col1': source_l,
    'col2': target_l,
    'col3': weight_l,
}

df_edges_l = pd.DataFrame(df_edges_l)
df_edges_l.rename(columns={'col1': 'Source', 'col2': 'Target', 'col3':'Weight'}, inplace=True)

df_edges_l

df_edges_l = df_edges_l.assign(Type='Undirected')
df_edges_l


,Source,Target,Weight,Type
0,years,voting,6,Undirected
1,years,rights,7,Undirected
2,march,voting,8,Undirected
3,march,rights,7,Undirected
4,voting,rights,209,Undirected
...,...,...,...,...
358,texas,bill,10,Undirected
359,voting,bills,8,Undirected
360,guest,essay,6,Undirected
361,manchin,voting,9,Undirected


#### Add word count as node attribute

In [38]:
words_right = right['title'].str.split().explode()
word_counts_r = words_right.value_counts().reset_index() # Calculate the word counts
word_counts_r.columns = ['word', 'count'] # Rename the columns
word_counts_r

,word,count
0,biden,380
1,voter,310
2,election,264
3,trump,245
4,identification,236
...,...,...
6007,wray,1
6008,seuss,1
6009,books,1
6010,pulled,1


In [39]:
words_left = left['title'].str.split().explode()
word_counts_l = words_left.value_counts().reset_index() # Calculate the word counts
word_counts_l.columns = ['Id', 'count'] # Rename the columns
word_counts_l

,Id,count
0,voting,507
1,voter,410
2,rights,255
3,identification,243
4,law,224
...,...,...
4175,basically,1
4176,bankruptcy,1
4177,jokes,1
4178,pottymouthed,1


In [40]:
filtered_word_counts_l = word_counts_l[word_counts_l['Id'].isin(df_edges_l['Source']) | word_counts_l['Id'].isin(df_edges_l['Target'])]
len(filtered_word_counts_l)

146

In [41]:
words_right = right['title'].str.split().explode()
word_counts_r = words_right.value_counts().reset_index() # Calculate the word counts
word_counts_r.columns = ['Id', 'count'] # Rename the columns
word_counts_r

,Id,count
0,biden,380
1,voter,310
2,election,264
3,trump,245
4,identification,236
...,...,...
6007,wray,1
6008,seuss,1
6009,books,1
6010,pulled,1


In [42]:
filtered_word_counts_r = word_counts_r[word_counts_r['Id'].isin(df_edges_r['Source']) | word_counts_r['Id'].isin(df_edges_r['Target'])]
len(filtered_word_counts_r)

690

## Shifterator

In [50]:
nodes_df['score'] = abs(nodes_df['score']) # convert shifterator scores all positive
nodes_df['score'] = nodes_df['score'] * 100
nodes_df

,score,word,left_right
0,0.000528,edifying,right
1,0.000653,bloch,right
2,0.000375,pageantry,left
3,0.000326,alleghany,right
4,0.000125,interbreeding,left
...,...,...,...
48917,0.000354,whitfield,right
48918,0.000326,barrabi,right
48919,0.000125,bushfire,left
48920,0.002772,dive,right


In [57]:
nodes_df['score'] = nodes_df['score'] * 100
nodes_df


,score,word,left_right
0,0.052796,edifying,right
1,0.065298,bloch,right
2,0.037506,pageantry,left
3,0.032649,alleghany,right
4,0.012502,interbreeding,left
...,...,...,...
48917,0.035437,whitfield,right
48918,0.032649,barrabi,right
48919,0.012502,bushfire,left
48920,0.277202,dive,right


In [58]:
# we first remove words from the nodes list that do not appear in the edges list
nodes_df_l = nodes_df[nodes_df['word'].isin(df_edges_l['Source']) | nodes_df['word'].isin(df_edges_l['Target'])]
nodes_df_r = nodes_df[nodes_df['word'].isin(df_edges_r['Source']) | nodes_df['word'].isin(df_edges_r['Target'])]
# we then remove words from the edges list that do not appear in the nodes list
Edges_df_filtered_l = df_edges_l[df_edges_l['Source'].isin(nodes_df['word'])| df_edges_l['Target'].isin(nodes_df['word'])]
Edges_df_filtered_r = df_edges_r[df_edges_r['Source'].isin(nodes_df['word'])| df_edges_r['Target'].isin(nodes_df['word'])]
Edges_df_filtered_l


,Source,Target,Weight,Type
0,years,voting,6,Undirected
1,years,rights,7,Undirected
2,march,voting,8,Undirected
3,march,rights,7,Undirected
4,voting,rights,209,Undirected
...,...,...,...,...
358,texas,bill,10,Undirected
359,voting,bills,8,Undirected
360,guest,essay,6,Undirected
361,manchin,voting,9,Undirected


In [59]:
# Group the DataFrame by 'Source' and 'Target' and sum the 'Weight'
df_edges_l.groupby(['Source', 'Target']).agg({'Weight': 'sum'}).reset_index()
df_edges_r.groupby(['Source', 'Target']).agg({'Weight': 'sum'}).reset_index()

# The DataFrame now has a single row for each combination of 'Source' and 'Target' with the 'Weight' values summed
df_edges_l
df_edges_r

# FIGURE OUT HOW TO COMBINE SOURCE AND TARGET IF THEY ARE THE SAME


,Source,Target,Weight,Type
0,interview,jordan,15,Undirected
1,interview,house,8,Undirected
2,interview,senator,91,Undirected
3,house,interview,7,Undirected
4,senator,scott,8,Undirected
...,...,...,...,...
2434,gop,player,6,Undirected
2435,gop,accused,6,Undirected
2436,gop,shooting,6,Undirected
2437,gop,killing,6,Undirected


In [60]:
#nodes_df_filtered['Id'] = range(1, len(nodes_df_filtered) + 1) # create new ID column
nodes_df_l.rename(columns={'word': 'Id'}, inplace=True)
nodes_df_r.rename(columns={'word': 'Id'}, inplace=True)
nodes_df_r


C:\Users\2146806A\AppData\Local\Temp\ipykernel_23292\680586395.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nodes_df_l.rename(columns={'word': 'Id'}, inplace=True)
C:\Users\2146806A\AppData\Local\Temp\ipykernel_23292\680586395.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nodes_df_r.rename(columns={'word': 'Id'}, inplace=True)


,score,Id,left_right
27,4.707924,reform,right
40,9.811430,illegal,right
88,0.622490,disastrous,right
207,1.212509,fake,right
425,1.119234,latest,right
...,...,...,...
48247,25.044135,presidential,left
48392,0.050008,ferry,left
48719,12.134470,fraud,left
48738,0.284847,origins,right


Add counts as attributes

In [61]:
words_r = right['maintext'].str.split().explode()
words_l = left['maintext'].str.split().explode()

# Calculate the word counts
word_counts_r = words_r.value_counts().reset_index()
word_counts_l = words_l.value_counts().reset_index()

# Rename the columns
word_counts_r.columns = ['Id', 'count']
word_counts_l.columns = ['Id', 'count']

In [62]:
# Get unique words from 'source' and 'target' columns in the edgelist dataframe
unique_words_l = set(df_edges_l['Source']).union(set(df_edges_l['Target']))
unique_words_r = set(df_edges_r['Source']).union(set(df_edges_r['Target']))

# Filter the word counts dataframe based on unique words
filtered_word_counts_l = word_counts_l[word_counts_l['Id'].isin(unique_words_l)]
filtered_word_counts_r = word_counts_r[word_counts_r['Id'].isin(unique_words_r)]

In [63]:
nodes_df_l

,score,Id,left_right
103,5.158562,restrictive,left
372,2.306484,purge,left
727,9.994325,restrictions,left
1357,2.407029,brian,right
1532,21.713072,democratic,left
...,...,...,...
47826,12.724065,senate,right
47845,60.298971,voting,left
47913,0.649111,opposing,right
48719,12.134470,fraud,left


In [64]:
nodes_df_filtered_l = nodes_df_l[nodes_df_l['Id'].isin(unique_words_l)]
nodes_df_filtered_r = nodes_df_r[nodes_df_r['Id'].isin(unique_words_r)]

In [65]:
nodes_df_filtered_r = pd.merge(nodes_df_filtered_r, filtered_word_counts_r, on='Id', how='left')
nodes_df_filtered_l = pd.merge(nodes_df_filtered_l, filtered_word_counts_l, on='Id', how='left')
nodes_df_filtered_r

,score,Id,left_right,count
0,4.707924,reform,right,443.0
1,9.811430,illegal,right,981.0
2,0.622490,disastrous,right,60.0
3,1.212509,fake,right,274.0
4,1.119234,latest,right,416.0
...,...,...,...,...
668,25.044135,presidential,left,954.0
669,0.050008,ferry,left,5.0
670,12.134470,fraud,left,1223.0
671,0.284847,origins,right,66.0


In [66]:
nodes_df_filtered_l

,score,Id,left_right,count
0,5.158562,restrictive,left,607.0
1,2.306484,purge,left,175.0
2,9.994325,restrictions,left,1362.0
3,2.407029,brian,right,309.0
4,21.713072,democratic,left,4308.0
...,...,...,...,...
138,12.724065,senate,right,2952.0
139,60.298971,voting,left,13353.0
140,0.649111,opposing,right,130.0
141,12.134470,fraud,left,2831.0


In [67]:
nodes_df_filtered_l['count'].isna().sum()

1

In [68]:
nodes_df_filtered_r = nodes_df_filtered_r.dropna(subset=['count'])
nodes_df_filtered_r.head()

,score,Id,left_right,count
0,4.707924,reform,right,443.0
1,9.811430,illegal,right,981.0
2,0.622490,disastrous,right,60.0
3,1.212509,fake,right,274.0
4,1.119234,latest,right,416.0


In [69]:
nodes_df_filtered_l = nodes_df_filtered_l.dropna(subset=['count'])
nodes_df_filtered_l.head()

,score,Id,left_right,count
0,5.158562,restrictive,left,607.0
1,2.306484,purge,left,175.0
2,9.994325,restrictions,left,1362.0
3,2.407029,brian,right,309.0
4,21.713072,democratic,left,4308.0


In [70]:
len(nodes_df_filtered_l)

142

In [71]:
nodes_df_filtered_l

,score,Id,left_right,count
0,5.158562,restrictive,left,607.0
1,2.306484,purge,left,175.0
2,9.994325,restrictions,left,1362.0
3,2.407029,brian,right,309.0
4,21.713072,democratic,left,4308.0
...,...,...,...,...
138,12.724065,senate,right,2952.0
139,60.298971,voting,left,13353.0
140,0.649111,opposing,right,130.0
141,12.134470,fraud,left,2831.0


In [72]:
nodes_df_filtered_l = nodes_df_filtered_l[['Id', 'score', 'left_right', 'count']]

In [73]:
Edges_df_filtered_l

,Source,Target,Weight,Type
0,years,voting,6,Undirected
1,years,rights,7,Undirected
2,march,voting,8,Undirected
3,march,rights,7,Undirected
4,voting,rights,209,Undirected
...,...,...,...,...
358,texas,bill,10,Undirected
359,voting,bills,8,Undirected
360,guest,essay,6,Undirected
361,manchin,voting,9,Undirected


In [108]:
#save edgelist as csv file to import into Gephi
df_edges_l.to_csv('C:/Users/2146806A/OneDrive - University of Glasgow/University of Glasgow/Amsterdam Visit/GITHUB FILES/Data/edges_left_title.csv', index=False)
df_edges_r.to_csv('C:/Users/2146806A/OneDrive - University of Glasgow/University of Glasgow/Amsterdam Visit/GITHUB FILES/Data/edges_right_title.csv', index=False)

In [114]:
nodes_df_filtered_l.to_csv('C:/Users/2146806A/OneDrive - University of Glasgow/University of Glasgow/Amsterdam Visit/GITHUB FILES/Data/nodes_left_title.csv', index=False) #export to csv
nodes_df_filtered_r.to_csv('C:/Users/2146806A/OneDrive - University of Glasgow/University of Glasgow/Amsterdam Visit/GITHUB FILES/Data/nodes_right_title.csv', index=False) #export to csv